In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F0'
SS_VER = 'SS_VER_2_1'
CRYPTO_TARGET='TINYAES128C'
%matplotlib widget

In [2]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        print("SS_VER_2_1")
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        print("SimpleSerial")
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

SS_VER_2_1
INFO: Found ChipWhisperer😍


In [3]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
    print("cw.programmers.STM32FProgrammer")
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif "SAM4S" in PLATFORM or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None
    
prog = cw.programmers.STM32FProgrammer
import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        print("CW308_SAM4S clause")
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:
        # print("else clause")
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

cw.programmers.STM32FProgrammer
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 213001229                 to 233649883                
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 96000000                  to 29538459                 
scope.clock.adc_rate                     changed from 96000000.0                to 29538459.0   

In [11]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd mcu/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CW308_STM32F0 with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CW308_STM32F0 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
Compiling:
Compiling:
Compiling:
Compiling:
-en     simpleserial-aes.c ...
-en     .././simpleserial/simpleserial.c ...
-en     .././hal/hal.c ...
-en     .././hal//stm32f0/stm32f0_hal.c ...
.
.
.
Compiling:
Compiling:
.
-en     .././crypto/tiny-AES128-C/aes.c ...
Compiling:
-en     .././crypto/aes-independant.c ...
Assembling: .././hal//stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF

In [12]:
cw.program_target(scope, prog, "mcu/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))


Detected unknown STM32F ID: 0x440
Extended erase (0x44), this can take ten seconds or more
Attempting to program 6467 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6467 bytes


In [13]:
target.baud = 38400  # or 115200

from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 100
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    if text[0] & 0x01:
        text[0] = 0xFF
    else:
        text[0] = 0x00
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 

Capturing traces:   0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
msg = bytearray([0]*16) #simpleserial uses bytearrays
target.simpleserial_write('p', msg)
print(target.simpleserial_read('r', 16))

CWbytearray(b'7d f7 6b 0c 1a b8 99 b3 3e 42 f0 47 b9 1b 54 6f')


In [15]:
assert len(trace_array) == 100
print("✔️ OK to continue!")

✔️ OK to continue!


In [17]:
one_list = []
zero_list = []

for i in range(len(trace_array)):
    if textin_array[i][0] == 0x00:
        zero_list.append(trace_array[i])
    else:
        one_list.append(trace_array[i])

assert len(one_list) > len(zero_list)/2
assert len(zero_list) > len(one_list)/2

In [23]:
import numpy as np
one_avg = np.mean(one_list, axis=0)
zero_avg = np.mean(zero_list, axis=0)
diff = one_avg - zero_avg
cw.plot(diff)


:Curve   [x]   (y)

In [34]:
# Try setting multiple bytes to 0x00 and 0xFF.
target.baud = 38400  # or 115200

from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 100
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    if text[0] & 0x01:
        text[0] = 0xFF
    else:
        text[0] = 0x00
    if text[1] & 0x01:
        text[1] = 0xFF
    else:
        text[1] = 0x00
    if text[2] & 0x01:
        text[2] = 0xFF
    else:
        text[2] = 0x00
    if text[3] & 0x01:
        text[3] = 0xFF
    else:
        text[3] = 0x00
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 

one_list = []
zero_list = []

for i in range(len(trace_array)):
    if textin_array[i][0] == 0x00:
        zero_list.append(trace_array[i])
    else:
        one_list.append(trace_array[i])

assert len(one_list) > len(zero_list)/2
assert len(zero_list) > len(one_list)/2
import numpy as np
one_avg = np.mean(one_list, axis=0)
zero_avg = np.mean(zero_list, axis=0)
diff = one_avg - zero_avg
cw.plot(diff)


Capturing traces:   0%|          | 0/100 [00:00<?, ?it/s]

:Curve   [x]   (y)

In [39]:
# Try using smaller hamming weight differences. Is the spike still distinct? What about if you capture more traces?
target.baud = 38400  # or 115200

from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 500
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    if text[0] & 0x01:
        text[0] = 0xFF
    else:
        text[0] = 0x00
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 

one_list = []
zero_list = []

for i in range(len(trace_array)):
    if textin_array[i][0] == 0x00:
        zero_list.append(trace_array[i])
    else:
        one_list.append(trace_array[i])

assert len(one_list) > len(zero_list)/2
assert len(zero_list) > len(one_list)/2
import numpy as np
one_avg = np.mean(one_list, axis=0)
zero_avg = np.mean(zero_list, axis=0)
diff = one_avg - zero_avg
cw.plot(diff)


Capturing traces:   0%|          | 0/30 [00:00<?, ?it/s]

:Curve   [x]   (y)

In [42]:
# We focused on the first byte here. Try putting the difference plots for multiple different bytes on the same plot.
target.baud = 38400  # or 115200

from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 100
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    if text[15] & 0x01:
        text[15] = 0xFF
    else:
        text[15] = 0x00

    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 

one_list = []
zero_list = []

for i in range(len(trace_array)):
    if textin_array[i][15] == 0x00:
        zero_list.append(trace_array[i])
    else:
        one_list.append(trace_array[i])

assert len(one_list) > len(zero_list)/2
assert len(zero_list) > len(one_list)/2
import numpy as np
one_avg = np.mean(one_list, axis=0)
zero_avg = np.mean(zero_list, axis=0)
diff = one_avg - zero_avg
cw.plot(diff)


Capturing traces:   0%|          | 0/100 [00:00<?, ?it/s]

:Curve   [x]   (y)